In [1]:
import os
import sys
import json
import base64
import logging
import time
import csv
from pathlib import Path
from io import BytesIO
from datetime import datetime
from typing import List, Dict, Optional, Tuple, Any
from dataclasses import dataclass, field, asdict


In [2]:

# ============================================================================
# ГЛОБАЛЬНЫЕ ПЕРЕМЕННЫЕ
# ============================================================================
# Добавить корень проекта в PYTHONPATH
PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name != "Team_5_AI_Tutor":
    PROJECT_ROOT = PROJECT_ROOT.parents[1]
sys.path.insert(0, str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = DATA_DIR / "raw" / "cloud_ru"

# Создаём директории если не существуют
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Ячейка 1: Импорты загружены")
print(f"   📂 Корневая директория: {PROJECT_ROOT}")
print(f"   📂 Директория для результатов: {OUTPUT_DIR}")


✅ Ячейка 1: Импорты загружены
   📂 Корневая директория: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor
   📂 Директория для результатов: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor/data/raw/cloud_ru


In [7]:
import asyncio
from playwright.async_api import async_playwright
from pathlib import Path
import json
from datetime import datetime
import re
from urllib.parse import urlparse
from PIL import Image

class CloudRuDocsScraper:
    def __init__(self, output_dir=OUTPUT_DIR, hide_sidebar=True, 
                 scale_factor=0.75, jpeg_quality=80):
        """
        Args:
            scale_factor: Масштаб DPI (0.5 = 50%, 0.75 = 75%, 1.0 = 100%)
            jpeg_quality: Качество JPEG (1-100, рекомендуется 70-85)
        """
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.base_url = "https://cloud.ru/docs/tutorials-evolution/list/doc-contents"
        self.domain = "https://cloud.ru"
        self.hide_sidebar = hide_sidebar
        self.scale_factor = scale_factor
        self.jpeg_quality = jpeg_quality
        
    def sanitize_filename(self, url):
        parsed = urlparse(url)
        path = parsed.path.strip('/')
        filename = path.replace('/', '_')
        filename = re.sub(r'[<>:"|?*]', '', filename)
        if len(filename) > 200:
            filename = filename[:200]
        return filename or "index"
    
    def extract_section_info(self, url):
        parsed = urlparse(url)
        path_parts = parsed.path.strip('/').split('/')
        
        section_info = {
            'category': path_parts[2] if len(path_parts) >= 3 else None,
            'subcategory': path_parts[4] if len(path_parts) >= 5 else None,
            'topic': path_parts[5] if len(path_parts) >= 6 else None
        }
        return section_info
    
    async def collect_tutorial_links(self, page):
        print("Загружаем главную страницу...")
        await page.goto(self.base_url, wait_until='networkidle')
        await page.wait_for_selector('a', timeout=10000)
        
        links = await page.evaluate(f'''() => {{
            const domain = "{self.domain}";
            const links = Array.from(document.querySelectorAll('a'));
            return links
                .map(link => link.href)
                .filter(href => 
                    href && 
                    href.startsWith(domain) && 
                    href.includes('/docs/') &&
                    !href.includes('#')
                );
        }}''')
        
        unique_links = sorted(list(set(links)))
        print(f"Найдено уникальных ссылок: {len(unique_links)}")
        return unique_links
    
    async def hide_sidebar_elements(self, page):
        if not self.hide_sidebar:
            return
        
        await page.evaluate('''() => {
            function isInsideMainContent(el) {
                const main = document.querySelector('main');
                const article = document.querySelector('article');
                return (main && main.contains(el)) || (article && article.contains(el));
            }
            
            // Левое меню
            const navs = document.querySelectorAll('nav');
            navs.forEach(nav => {
                if (!nav.closest('main, article')) {
                    nav.style.display = 'none';
                }
            });
            
            // Сайдбары
            document.querySelectorAll('[class*="sidebar"]').forEach(el => {
                if (!isInsideMainContent(el)) el.style.display = 'none';
            });
            
            // Правая панель
            document.querySelectorAll('aside').forEach(aside => {
                if (!isInsideMainContent(aside)) aside.style.display = 'none';
            });
            
            // Верхний хедер
            document.querySelectorAll('header').forEach(header => {
                if (!isInsideMainContent(header)) header.style.display = 'none';
            });
            
            // Хлебные крошки
            document.querySelectorAll('[class*="breadcrumb"]').forEach(el => {
                el.style.display = 'none';
            });
            
            // Футер
            document.querySelectorAll('footer').forEach(f => f.style.display = 'none');
            
            // "Эта статья полезна?"
            document.querySelectorAll('*').forEach(el => {
                if (el.textContent.includes('Эта статья полезна') && 
                    el.textContent.length < 100) {
                    let parent = el.closest('div, section');
                    if (parent) parent.style.display = 'none';
                }
            });
            
            // Расширяем контент
            const main = document.querySelector('main');
            if (main) {
                main.style.marginLeft = '0';
                main.style.marginRight = '0';
                main.style.maxWidth = '100%';
                main.style.width = '100%';
                main.style.padding = '40px';
            }
        }''')
        
        await page.wait_for_timeout(500)
    
    async def get_page_title(self, page):
        try:
            title = await page.evaluate('''() => {
                const h1 = document.querySelector('h1');
                return h1 ? h1.textContent.trim() : '';
            }''')
            return title or "Без заголовка"
        except:
            return "Без заголовка"
    
    async def expand_all_containers(self, page):
        """Разворачивает ВСЕ контейнеры на полную высоту"""
        result = await page.evaluate("""
            async () => {
                document.documentElement.style.height = 'auto';
                document.documentElement.style.overflow = 'visible';
                document.body.style.height = 'auto';
                document.body.style.minHeight = '100%';
                document.body.style.overflow = 'visible';
                
                const appWrapper = document.querySelector('.styles_app_wrapper__Xfzgu');
                if (appWrapper) {
                    appWrapper.style.height = 'auto';
                    appWrapper.style.minHeight = '100%';
                    appWrapper.style.overflow = 'visible';
                }
                
                const layoutWrapper = document.querySelector('.styles_layout__PDzxx');
                if (layoutWrapper) {
                    layoutWrapper.style.height = 'auto';
                    layoutWrapper.style.overflow = 'visible';
                }
                
                const main = document.querySelector('main');
                if (main) {
                    main.style.height = 'auto';
                    main.style.overflow = 'visible';
                }
                
                const wrappers = document.querySelectorAll(
                    '.styles_page_wrapper__FHeMA, ' +
                    '.styles_sectionWrapper__tHdBi, ' +
                    '.styles_contentWrapper__LTcY9, ' +
                    '.styles_article_wrapper__3p7lX, ' +
                    '.styles_content_with_bread_crumbs__4pp9u'
                );
                
                wrappers.forEach(wrapper => {
                    wrapper.style.height = 'auto';
                    wrapper.style.maxHeight = 'none';
                    wrapper.style.overflow = 'visible';
                });
                
                const scrollContainer = document.querySelector('[data-overlayscrollbars-viewport]');
                const contentContainer = document.querySelector('[data-overlayscrollbars-contents]');
                
                if (scrollContainer) {
                    scrollContainer.style.height = 'auto';
                    scrollContainer.style.maxHeight = 'none';
                    scrollContainer.style.overflow = 'visible';
                    scrollContainer.style.position = 'static';
                }
                
                if (contentContainer) {
                    contentContainer.style.transform = 'none';
                    contentContainer.style.height = 'auto';
                    contentContainer.style.position = 'static';
                    contentContainer.style.top = '0';
                    contentContainer.style.left = '0';
                    contentContainer.style.margin = '0';
                    contentContainer.style.padding = '0';
                }
                
                const articleScroll = document.querySelector('.styles_article_scroll__ELGYB');
                if (articleScroll) {
                    articleScroll.style.height = 'auto';
                    articleScroll.style.maxHeight = 'none';
                    articleScroll.style.overflow = 'visible';
                }
                
                await new Promise(resolve => setTimeout(resolve, 1000));
                
                const finalHeight = Math.max(
                    document.body.scrollHeight,
                    document.documentElement.scrollHeight,
                    main ? main.scrollHeight : 0
                );
                
                return {
                    success: true,
                    bodyHeight: document.body.scrollHeight,
                    htmlHeight: document.documentElement.scrollHeight,
                    mainHeight: main ? main.scrollHeight : 0,
                    finalHeight: finalHeight
                };
            }
        """)
        return result
    
    def compress_image(self, png_path):
        """Конвертирует PNG в JPEG с заданным качеством"""
        try:
            # Открываем PNG
            with Image.open(png_path) as img:
                # Конвертируем в RGB (JPEG не поддерживает прозрачность)
                if img.mode in ('RGBA', 'LA', 'P'):
                    background = Image.new('RGB', img.size, (255, 255, 255))
                    if img.mode == 'P':
                        img = img.convert('RGBA')
                    background.paste(img, mask=img.split()[-1] if img.mode == 'RGBA' else None)
                    img = background
                elif img.mode != 'RGB':
                    img = img.convert('RGB')
                
                # Сохраняем как JPEG
                jpeg_path = png_path.with_suffix('.jpg')
                img.save(
                    jpeg_path,
                    'JPEG',
                    quality=self.jpeg_quality,
                    optimize=True
                )
                
                original_size = png_path.stat().st_size
                compressed_size = jpeg_path.stat().st_size
                
                # Удаляем PNG
                png_path.unlink()
                
                return {
                    'original_size': original_size,
                    'compressed_size': compressed_size,
                    'compression_ratio': compressed_size / original_size,
                    'path': str(jpeg_path),
                    'filename': jpeg_path.name
                }
        except Exception as e:
            print(f"    ⚠️ Ошибка сжатия: {e}")
            return None
    
    async def take_full_page_screenshot(self, page, url, filename):
        """Создает полностраничный скриншот с оптимизацией"""
        try:
            await page.goto(url, wait_until='networkidle', timeout=60000)
            await page.wait_for_timeout(2000)
            
            await self.hide_sidebar_elements(page)
            await page.wait_for_timeout(1000)
            
            page_title = await self.get_page_title(page)
            section_info = self.extract_section_info(url)
            
            print(f"  Разворачиваем контейнеры...")
            expand_result = await self.expand_all_containers(page)
            
            print(f"  ✓ Высота: {expand_result.get('finalHeight')}px")
            
            # Сохраняем PNG
            png_path = self.output_dir / f"{filename}.png"
            
            await page.screenshot(
                path=str(png_path),
                full_page=True
            )
            
            # Конвертируем в JPEG
            print(f"  Сжимаем изображение (качество={self.jpeg_quality})...")
            compression_result = self.compress_image(png_path)
            
            if compression_result:
                original_mb = compression_result['original_size'] / 1024 / 1024
                compressed_mb = compression_result['compressed_size'] / 1024 / 1024
                ratio = compression_result['compression_ratio'] * 100
                
                # Получаем размеры
                with Image.open(compression_result['path']) as img:
                    width, height = img.size
                
                print(f"  ✓ {page_title}")
                print(f"  ✓ Размер: {width}x{height}px")
                print(f"  ✓ Файл: {original_mb:.1f}MB → {compressed_mb:.1f}MB ({ratio:.0f}%)\n")
                
                return {
                    'success': True,
                    'mode': 'playwright_full_page_compressed',
                    'title': page_title,
                    'section_info': section_info,
                    'width': width,
                    'height': height,
                    'original_size_mb': original_mb,
                    'compressed_size_mb': compressed_mb,
                    'compression_ratio': ratio,
                    'screenshot': {
                        'path': compression_result['path'],
                        'filename': compression_result['filename']
                    }
                }
            else:
                return {'success': False, 'error': 'Compression failed'}
                
        except Exception as e:
            print(f"  ✗ Ошибка: {e}")
            import traceback
            traceback.print_exc()
            return {'success': False, 'error': str(e)}
    
    async def scrape_all(self, max_pages=None):
        start_time = datetime.now()
        results = []
        
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            
            # device_scale_factor влияет на DPI
            context = await browser.new_context(
                viewport={'width': 1920, 'height': 1080},
                device_scale_factor=self.scale_factor  # 0.75 = 75% качества
            )
            page = await context.new_page()
            
            links = await self.collect_tutorial_links(page)
            if max_pages:
                links = links[:max_pages]
            
            print(f"\n{'='*60}")
            print(f"Скриншоты для {len(links)} страниц")
            print(f"DPI: {self.scale_factor*100:.0f}%, JPEG качество: {self.jpeg_quality}")
            print(f"{'='*60}\n")
            
            for i, url in enumerate(links, 1):
                print(f"[{i}/{len(links)}] {url}")
                
                filename = self.sanitize_filename(url)
                screenshot_result = await self.take_full_page_screenshot(page, url, filename)
                
                result = {
                    'index': i,
                    'url': url,
                    'filename': filename,
                    'timestamp': datetime.now().isoformat(),
                    **screenshot_result
                }
                
                results.append(result)
                await asyncio.sleep(1.5)
            
            await browser.close()
        
        total_original = sum(r.get('original_size_mb', 0) for r in results if r['success'])
        total_compressed = sum(r.get('compressed_size_mb', 0) for r in results if r['success'])
        
        metadata = {
            'scrape_date': start_time.isoformat(),
            'duration_seconds': (datetime.now() - start_time).total_seconds(),
            'total_pages': len(links),
            'successful': len([r for r in results if r['success']]),
            'method': 'playwright_full_page_compressed',
            'scale_factor': self.scale_factor,
            'jpeg_quality': self.jpeg_quality,
            'sidebar_hidden': self.hide_sidebar,
            'total_size_mb': {
                'original': total_original,
                'compressed': total_compressed,
                'saved': total_original - total_compressed
            },
            'pages': results
        }
        
        metadata_path = self.output_dir / 'metadata.json'
        with open(metadata_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        
        print(f"\n{'='*60}")
        print(f"Готово! {metadata['successful']}/{metadata['total_pages']}")
        print(f"Экономия: {total_original:.1f}MB → {total_compressed:.1f}MB")
        print(f"Время: {metadata['duration_seconds']:.1f} сек")
        print(f"{'='*60}\n")
        
        return metadata

In [8]:
# ============================================================
# ЗАПУСК С РАЗНЫМИ НАСТРОЙКАМИ
# ============================================================

# Вариант 1: Средняя оптимизация (рекомендуется)
scraper = CloudRuDocsScraper(
    output_dir=OUTPUT_DIR,
    hide_sidebar=True,
    scale_factor=0.75,    # 75% DPI (1440px вместо 1920px)
    jpeg_quality=70       # Качество JPEG 70%
)

# Вариант 2: Максимальная оптимизация
# scraper = CloudRuDocsScraper(
#     output_dir=OUTPUT_DIR,
#     hide_sidebar=True,
#     scale_factor=0.5,     # 50% DPI (960px вместо 1920px)
#     jpeg_quality=75       # Качество JPEG 75%
# )

# Вариант 3: Минимальная оптимизация
# scraper = CloudRuDocsScraper(
#     output_dir=OUTPUT_DIR,
#     hide_sidebar=True,
#     scale_factor=1.0,     # 100% DPI
#     jpeg_quality=85       # Качество JPEG 85%
# )

# metadata = await scraper.scrape_all(max_pages=3)
# Полный запуск
metadata = await scraper.scrape_all()

Загружаем главную страницу...
Найдено уникальных ссылок: 120

Скриншоты для 120 страниц
DPI: 75%, JPEG качество: 70

[1/120] https://cloud.ru/docs/
  Разворачиваем контейнеры...
  ✓ Высота: 6557px
  Сжимаем изображение (качество=70)...
  ✓ Документация  Cloud.ru Evolution
  ✓ Размер: 1500x4918px
  ✓ Файл: 0.5MB → 0.3MB (75%)

[2/120] https://cloud.ru/docs/evolution/overview/contacts/technical-support
  Разворачиваем контейнеры...
  ✓ Высота: 1589px
  Сжимаем изображение (качество=70)...
  ✓ Обратиться в поддержку
  ✓ Размер: 1500x1192px
  ✓ Файл: 0.1MB → 0.1MB (92%)

[3/120] https://cloud.ru/docs/tutorials-evolution/list/index?source-platform=Evolution
  Разворачиваем контейнеры...
  ✓ Высота: 6873px
  Сжимаем изображение (качество=70)...
  ✓ Практические руководства Cloud.ru Evolution
  ✓ Размер: 1500x5155px
  ✓ Файл: 0.5MB → 0.4MB (78%)

[4/120] https://cloud.ru/docs/tutorials-evolution/list/topics/arenadata-db__adbc-backup?source-platform=Evolution
  Разворачиваем контейнеры...
  ✓ 

Traceback (most recent call last):
  File "/tmp/ipykernel_2673518/547093768.py", line 275, in take_full_page_screenshot
    await page.goto(url, wait_until='networkidle', timeout=60000)
  File "/home/gna/anaconda3/envs/rocm/lib/python3.10/site-packages/playwright/async_api/_generated.py", line 9041, in goto
    await self._impl_obj.goto(
  File "/home/gna/anaconda3/envs/rocm/lib/python3.10/site-packages/playwright/_impl/_page.py", line 552, in goto
    return await self._main_frame.goto(**locals_to_params(locals()))
  File "/home/gna/anaconda3/envs/rocm/lib/python3.10/site-packages/playwright/_impl/_frame.py", line 153, in goto
    await self._channel.send(
  File "/home/gna/anaconda3/envs/rocm/lib/python3.10/site-packages/playwright/_impl/_connection.py", line 69, in send
    return await self._connection.wrap_api_call(
  File "/home/gna/anaconda3/envs/rocm/lib/python3.10/site-packages/playwright/_impl/_connection.py", line 559, in wrap_api_call
    raise rewrite_error(error, f"{parse

[39/120] https://cloud.ru/docs/tutorials-evolution/list/topics/free-tier-vm__nextcloud?source-platform=Evolution
  Разворачиваем контейнеры...
  ✓ Высота: 6262px
  Сжимаем изображение (качество=70)...
  ✓ Организация хранения файлов через Nextcloud с доступом через веб-интерфейс и мобильное приложение
  ✓ Размер: 1500x4697px
  ✓ Файл: 0.5MB → 0.4MB (75%)

[40/120] https://cloud.ru/docs/tutorials-evolution/list/topics/free-tier-vm__outline?source-platform=Evolution
  Разворачиваем контейнеры...
  ✓ Высота: 11390px
  Сжимаем изображение (качество=70)...
  ✓ Развертывание Wiki-сервиса Outline на виртуальной машине
  ✓ Размер: 1500x8543px
  ✓ Файл: 0.8MB → 0.6MB (71%)

[41/120] https://cloud.ru/docs/tutorials-evolution/list/topics/free-tier-vm__passbolt?source-platform=Evolution
  Разворачиваем контейнеры...
  ✓ Высота: 6224px
  Сжимаем изображение (качество=70)...
  ✓ Развертывание личного менеджера паролей на базе PassBolt на виртуальной машине
  ✓ Размер: 1500x4668px
  ✓ Файл: 0.5MB → 0